# Training machine learning models on pairs of substrates in individual organisms

## Imports

In [1]:
from subpred.util import load_df
from subpred.graph import preprocess_data, get_substrate_matrix
from subpred.pssm import calculate_pssm_feature
from subpred.compositions import calculate_aac, calculate_paac
import pandas as pd
from subpred.cdhit import cd_hit
from sklearn.pipeline import make_pipeline
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.feature_selection import SelectPercentile
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.metrics import classification_report
from subpred.custom_transformers import FeatureCombinator, get_feature_type_combinations

from subpred.util import load_df
import networkx as nx


## Functions

### Dataset

In [2]:
def get_classification_task(
    organism_ids: set,
    labels: set,
    clustering_threshold: int = None,
    dataset_folder_path: str = "../data/datasets",
) -> pd.DataFrame:
    # TODO handling for multi-substrate
    # TODO ability to use go terms or chebi terms (compare sample count, performance)

    (
        df_uniprot,
        df_uniprot_goa,
        graph_go_filtered,
        graph_chebi_filtered,
    ) = preprocess_data(
        organism_ids=organism_ids, datasets_folder_path=dataset_folder_path
    )
    # TODO go through method code
    df_substrate_overlaps, dict_chebi_to_uniprot = get_substrate_matrix(
        datasets_folder_path=dataset_folder_path,
        graph_chebi=graph_chebi_filtered,
        graph_go=graph_go_filtered,
        df_uniprot_goa=df_uniprot_goa,
        min_overlap=0,
        max_overlap=int(1e6),
    )
    assert df_substrate_overlaps.shape[0] == len(dict_chebi_to_uniprot.keys())
    chebi_name_to_term = {
        name: term for term, name in graph_chebi_filtered.nodes(data="name")
    }
    chebi_term_to_name = {
        term: name for term, name in graph_chebi_filtered.nodes(data="name")
    }
    molecule_counts = {
        chebi_term_to_name[term]: len(proteins)
        for term, proteins in dict_chebi_to_uniprot.items()
    }
    # print(sorted(molecule_counts.items(), key=lambda item: item[1], reverse=True))

    protein_to_label = list()
    for label in labels:
        label_proteins = dict_chebi_to_uniprot[chebi_name_to_term[label]]
        for protein in label_proteins:
            protein_to_label.append([protein, label])

    df_labels = pd.DataFrame.from_records(
        protein_to_label, columns=["Uniprot", "label"], index="Uniprot"
    )

    df_labels = df_labels[~df_labels.index.duplicated()]  # TODO series?
    df_sequences = df_uniprot.loc[df_labels.index].sequence.to_frame()
    if clustering_threshold:
        cluster_representatives = cd_hit(
            df_sequences.sequence, identity_threshold=clustering_threshold
        )
        df_sequences = df_sequences.loc[cluster_representatives]
        df_labels = df_labels.loc[cluster_representatives]
    return pd.concat([df_sequences, df_labels], axis=1)


### Features

In [3]:
def get_features(series_sequences: pd.Series):
    df_aac = calculate_aac(series_sequences)
    df_paac = calculate_paac(series_sequences)
    df_pssm_50_1 = calculate_pssm_feature(
        series_sequences,
        tmp_folder="../data/intermediate/blast/pssm_uniref50_1it",
        blast_db="../data/raw/uniref/uniref50/uniref50.fasta",
        iterations=1,
        psiblast_threads=-1,
        verbose=False,
        feature_name="PSSM_50_1",
    )
    df_pssm_50_3 = calculate_pssm_feature(
        series_sequences,
        tmp_folder="../data/intermediate/blast/pssm_uniref50_3it",
        blast_db="../data/raw/uniref/uniref50/uniref50.fasta",
        iterations=3,
        psiblast_threads=-1,
        verbose=False,
        feature_name="PSSM_50_3",
    )
    df_pssm_90_1 = calculate_pssm_feature(
        series_sequences,
        tmp_folder="../data/intermediate/blast/pssm_uniref90_3it",
        blast_db="../data/raw/uniref/uniref90/uniref90.fasta",
        iterations=1,
        psiblast_threads=-1,
        verbose=False,
        feature_name="PSSM_90_1",
    )
    df_pssm_90_3 = calculate_pssm_feature(
        series_sequences,
        tmp_folder="../data/intermediate/blast/pssm_uniref90_3it",
        blast_db="../data/raw/uniref/uniref90/uniref90.fasta",
        iterations=3,
        psiblast_threads=-1,
        verbose=False,
        feature_name="PSSM_90_3",
    )
    df_features = pd.concat(
        [
            df_aac,
            df_paac,
            df_pssm_50_1,
            df_pssm_50_3,
            df_pssm_90_1,
            df_pssm_90_3,
        ],
        axis=1,
    )
    return df_features


### Eval

In [4]:
list(range(2, 101, 2))

[2,
 4,
 6,
 8,
 10,
 12,
 14,
 16,
 18,
 20,
 22,
 24,
 26,
 28,
 30,
 32,
 34,
 36,
 38,
 40,
 42,
 44,
 46,
 48,
 50,
 52,
 54,
 56,
 58,
 60,
 62,
 64,
 66,
 68,
 70,
 72,
 74,
 76,
 78,
 80,
 82,
 84,
 86,
 88,
 90,
 92,
 94,
 96,
 98,
 100]

In [5]:
def get_xy(df_dataset, df_features):
    # converting data to numpy
    label_encoder = LabelEncoder()
    label_encoder.fit(sorted(df_dataset.label.unique()))
    class_labels = label_encoder.classes_
    sample_names = df_features.index.values
    feature_names = df_features.columns.values
    X = df_features.values
    y = label_encoder.transform(df_dataset.label)
    # train test eval split
    (
        X_train,
        X_eval,
        y_train,
        y_eval,
        sample_names_train,
        sample_names_eval,
    ) = train_test_split(X, y, sample_names, test_size=0.2, random_state=1, stratify=y)
    return (
        X_train,
        X_eval,
        y_train,
        y_eval,
        sample_names_train,
        sample_names_eval,
        feature_names,
        class_labels
    )


def get_model(X_train, y_train, feature_names: pd.Series, type: str = "plain"):
    # types: featurecomb, anova, featurecomb_anova, plain TODO
    feature_type_combinations = get_feature_type_combinations(
        feature_names=feature_names
    )

    param_grid = dict()
    pipeline_elements = list()
    pipeline_elements.append(StandardScaler())
    if type in ["featurecomb", "featurecomb_anova"]:
        pipeline_elements.append(FeatureCombinator(feature_names=feature_names))
        param_grid.update(
            {"featurecombinator__feature_types": feature_type_combinations}
        )
    elif type in ["anova", "featurecomb_anova"]:
        pipeline_elements.append(SelectPercentile())
        param_grid.update(
            {
                "selectpercentile__percentile": list(range(2, 101, 2)),
            }
        )  # TODO all?
    pipeline_elements.append(SVC(random_state=1, probability=True))
    param_grid.update(
        {
            "svc__C": [0.1, 1, 10],
            # "svc__gamma": ["scale", "auto"],
            "svc__class_weight": ["balanced", None],
        }
    )
    pipeline = make_pipeline(*pipeline_elements)
    print(pipeline)
    print(param_grid)

    # hyperparam optim & crossval
    gridsearch = GridSearchCV(
        estimator=pipeline,
        param_grid=param_grid,
        # scoring=["f1", "precision", "recall"],
        # refit="f1",
        scoring="f1",
        refit=True,
        cv=5,
        n_jobs=-1,
        return_train_score=True,
        # verbose=20
    )
    gridsearch.fit(X_train, y_train)
    return gridsearch


def eval(X_eval, y_eval, model):
    y_pred = model.predict(X_eval)
    return classification_report(y_true=y_eval, y_pred=y_pred, output_dict=True)


In [6]:


# def evaluate(df_dataset, df_features):
#     # converting data to numpy
#     label_encoder = LabelEncoder()
#     label_encoder.fit(sorted(df_dataset.label.unique()))
#     sample_names = df_features.index.values
#     feature_names = df_features.columns.values
#     X = df_features.values
#     y = label_encoder.transform(df_dataset.label)
#     # train test eval split
#     (
#         X_train,
#         X_eval,
#         y_train,
#         y_eval,
#         sample_names_train,
#         sample_names_eval,
#     ) = train_test_split(X, y, sample_names, test_size=0.2, random_state=1, stratify=y)

#     feature_type_combinations = get_feature_type_combinations(
#         feature_names=feature_names
#     )
#     feature_combinator = FeatureCombinator(feature_names=feature_names)
#     model = make_pipeline(
#         StandardScaler(), feature_combinator, SVC(random_state=1, probability=True)
#     )
#     param_grid = {
#         "svc__C": [0.1, 1, 10],
#         # "svc__gamma": ["scale", "auto"],
#         "svc__class_weight": ["balanced", None],
#         "featurecombinator__feature_types": feature_type_combinations,
#         # "selectpercentile__percentile": list(range(1, 101, 5)),
#     }

#     # hyperparam optim & crossval
#     gridsearch = GridSearchCV(
#         estimator=model,
#         param_grid=param_grid,
#         scoring=["f1", "precision", "recall"],
#         refit="f1",
#         cv=5,
#         n_jobs=-1,
#         return_train_score=True,
#         # verbose=20
#     )
#     gridsearch.fit(X_train, y_train)
#     print("Best train score:", gridsearch.best_score_)
#     print("Best train params:", gridsearch.best_params_)
#     gridsearch_results = pd.DataFrame.from_dict(gridsearch.cv_results_)
#     model_optim = gridsearch.best_estimator_

#     # eval
#     y_pred = model_optim.predict(X_eval)
#     print(classification_report(y_true=y_eval, y_pred=y_pred))
#     # print(model_optim.predict_proba(X_eval))  # TODO compare with actual labels
#     return gridsearch_results

## New dataset creation

In [7]:
def get_id_update_dict(graph, field="alt_id"):
    dict_update_id = dict()
    for node, alt_ids in graph.nodes(data=field):
        if not alt_ids:
            continue
        for alt_id in alt_ids:
            dict_update_id[alt_id] = node

    return dict_update_id


def get_go_subgraph(
    root_term_name: str, edge_keys: set = {"is_a"}, aspects_filter: set = None
):
    # edge_keys: {"is_a"}, aspects_filter: {"molecular_function"}
    graph_go = load_df("go_obo")
    go_name_to_id = {name: node for node, name in list(graph_go.nodes(data="name"))}
    root_term_id = go_name_to_id[root_term_name]

    root_term_descendants = nx.ancestors(graph_go, root_term_id)
    graph_go_sub = graph_go.subgraph(root_term_descendants)

    if edge_keys:
        graph_go_sub = graph_go_sub.edge_subgraph(
            [
                (node1, node2, key)
                for node1, node2, key in graph_go_sub.edges(keys=True)
                if key in edge_keys
            ]
        )
    if aspects_filter:
        nodes_aspects = {
            node
            for node, namespace in graph_go.nodes(data="namespace")
            if namespace in aspects_filter
        }
        graph_go_sub = graph_go_sub.subgraph(nodes_aspects)

    return graph_go_sub.copy()


# TODO turn this into new dataset creation pipeline. 
# first, get protein dataset
# then, get goa annotations for transmembrane transporters
# annotate with chebi terms (primary only)
def get_goa_subset(
    root_term_name: str,
    proteins_filter: set = None,
    qualifiers_filter: set = None,
    aspects_filter: set = None,
    evidence_codes_exclude: set = None,
):
    # creates subset of go annotations below a root node, and filtered by a set of proteins.
    graph_go = load_df("go_obo")

    go_name_to_id = {name: node for node, name in list(graph_go.nodes(data="name"))}
    go_id_to_name = {node: name for node, name in list(graph_go.nodes(data="name"))}
    root_term_id = go_name_to_id[root_term_name]

    # use is_a key for retrieving ancestors and descendants, to avoid crossing into different aspect
    graph_go_isa = graph_go.edge_subgraph(
        [
            (node1, node2, key)
            for node1, node2, key in graph_go.edges(keys=True)
            if key == "is_a"
        ]
    )
    root_term_descendants = nx.ancestors(graph_go_isa, root_term_id)

    df_goa = load_df("go")

    # update go terms
    dict_update_ids = get_id_update_dict(graph_go, field="alt_id")
    df_goa.go_id.apply(
        lambda id: dict_update_ids[id] if id in dict_update_ids.keys() else id
    )
    # filter annotations
    df_goa = df_goa[df_goa.go_id.isin(root_term_descendants)]
    if proteins_filter:
        df_goa = df_goa[df_goa.Uniprot.isin(proteins_filter)]
    if qualifiers_filter:
        df_goa = df_goa[df_goa.qualifier.isin(qualifiers_filter)]
    if aspects_filter:
        df_goa = df_goa[df_goa.aspect.isin(aspects_filter)]
    if evidence_codes_exclude:
        df_goa = df_goa[~df_goa.evidence_code.isin(evidence_codes_exclude)]

    df_goa = df_goa.drop_duplicates().reset_index(drop=True)
    df_goa["go_term"] = df_goa.go_id.map(go_id_to_name)
    df_goa["ancestors"] = df_goa.go_id.apply(
        lambda x: set(nx.descendants(graph_go_isa, x) & root_term_descendants)
    )
    df_goa = df_goa.explode("ancestors").reset_index(drop=True)
    df_goa["ancestor_term"] = df_goa.ancestors.map(go_id_to_name)

    return df_goa


## Main

In [8]:
# TODO svm percentages or other confidence values return
# TODO determinism
# TODO compare different models
    # Only one pssm
    # Feature selector
    # All features with percentage
# TODO parameters for adjusting model
# TODO separate functions for dataset splitting, model creation and evaluation

In [9]:
labels = {"potassium(1+)", "calcium(2+)"}

dataset_name_to_organism_ids = {
    "human": {9606},
    "athaliana": {3702},
    "ecoli": {83333},
    "yeast": {559292},
}
dataset_name_to_organism_ids["all"] = {
    list(s)[0] for s in dataset_name_to_organism_ids.values() if len(s) == 1
}


test_cases = [
    ("athaliana", "potassium(1+)", "calcium(2+)"),
    ("athaliana", "inorganic anion", "inorganic cation"),
    ("athaliana", "carboxylic acid anion", "inorganic anion"),
    # ("ecoli", "carbohydrate derivative", "monosaccharide"),
    # ("ecoli", "monocarboxylic acid", "amino acid"),
    # ("human", "calcium(2+)", "sodium(1+)"),
    # ("human", "calcium(2+)", "potassium(1+)"),
    # ("human", "sodium(1+)", "potassium(1+)"),
    # ("human", "inorganic anion", "inorganic cation"),
    # ("yeast", "amide", "amino acid derivative"),
]

# TODO P05556 protein and others do not belong in there? Should be solved with new pipeline

for dataset_name, substrate1, substrate2 in test_cases:
    print(dataset_name, substrate1, substrate2)
    organism_ids = dataset_name_to_organism_ids[dataset_name]
    df_dataset = get_classification_task(
        organism_ids=organism_ids,
        labels={substrate1, substrate2},
        clustering_threshold=70,
    )
    print(df_dataset.label.value_counts())

    # TODO this is a quickfix, redo pipeline
    tmtp_proteins = get_goa_subset(
        root_term_name="transmembrane transporter activity",
        qualifiers_filter=["enables"],
        aspects_filter=["F"],
        proteins_filter=df_dataset.index.tolist(),
    ).Uniprot.unique()
    df_dataset = df_dataset[df_dataset.index.isin(set(tmtp_proteins))]
    # TODO quickfix end

    df_features = get_features(df_dataset.sequence)
    df_features = df_features.loc[df_features.index.sort_values()]
    df_dataset = df_dataset.loc[df_features.index]
    df_features = df_features.loc[df_dataset.index]

    # print(df_dataset.shape, df_features.shape)
    # TODO get more eval values from cv, like performance on individual classes
    # TODO decide which measures to return, put them into a dataframe to print
    # TODO compare different models
    # df_cv_results = evaluate(df_dataset=df_dataset, df_features=df_features)
    (
        X_train,
        X_eval,
        y_train,
        y_eval,
        sample_names_train,
        sample_names_eval,
        feature_names,
        class_labels
    ) = get_xy(df_dataset=df_dataset, df_features=df_features)

    # TODO ability to get different models
    model = get_model(X_train=X_train, y_train=y_train, feature_names=feature_names)
    df_cv_results = pd.DataFrame.from_dict(model.cv_results_)
    eval_results = eval(X_eval=X_eval, y_eval=y_eval, model=model.best_estimator_)
    display(df_cv_results)
    # TODO precrec or roc-auc curves? either from estimator or from predictions. compare pos and neg

athaliana potassium(1+) calcium(2+)


cd-hit: clustered 98 sequences into 63 clusters at threshold 70
label
potassium(1+)    34
calcium(2+)      29
Name: count, dtype: int64
Pipeline(steps=[('standardscaler', StandardScaler()),
                ('svc', SVC(probability=True, random_state=1))])
{'svc__C': [0.1, 1, 10], 'svc__class_weight': ['balanced', None]}


,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_svc__C,param_svc__class_weight,params,split0_test_score,split1_test_score,split2_test_score,...,mean_test_score,std_test_score,rank_test_score,split0_train_score,split1_train_score,split2_train_score,split3_train_score,split4_train_score,mean_train_score,std_train_score
0,0.016103,0.001800,0.005786,0.000948,0.1,balanced,"{'svc__C': 0.1, 'svc__class_weight': 'balanced'}",0.000000,0.750000,0.444444,...,0.553175,0.333632,6,0.000000,1.0,1.0,1.0,0.975610,0.795122,0.397673
1,0.016611,0.002565,0.004632,0.000884,0.1,None,"{'svc__C': 0.1, 'svc__class_weight': None}",0.750000,0.666667,0.666667,...,0.692857,0.034007,5,0.677966,0.7,0.7,0.7,0.688525,0.693298,0.008861
2,0.014580,0.003685,0.003672,0.001296,1,balanced,"{'svc__C': 1, 'svc__class_weight': 'balanced'}",0.666667,0.750000,0.444444,...,0.750000,0.190840,4,1.000000,1.0,1.0,1.0,1.000000,1.000000,0.000000
3,0.011945,0.003023,0.004218,0.001264,1,None,"{'svc__C': 1, 'svc__class_weight': None}",0.666667,0.888889,0.400000,...,0.772929,0.216294,1,1.000000,1.0,1.0,1.0,1.000000,1.000000,0.000000
4,0.014192,0.001769,0.004726,0.000153,10,balanced,"{'svc__C': 10, 'svc__class_weight': 'balanced'}",0.666667,0.750000,0.400000,...,0.763333,0.225191,2,1.000000,1.0,1.0,1.0,1.000000,1.000000,0.000000
5,0.011775,0.001589,0.003628,0.000493,10,None,"{'svc__C': 10, 'svc__class_weight': None}",0.666667,0.750000,0.400000,...,0.763333,0.225191,2,1.000000,1.0,1.0,1.0,1.000000,1.000000,0.000000


athaliana inorganic anion inorganic cation
cd-hit: clustered 312 sequences into 199 clusters at threshold 70
label
inorganic cation    158
inorganic anion      41
Name: count, dtype: int64
Pipeline(steps=[('standardscaler', StandardScaler()),
                ('svc', SVC(probability=True, random_state=1))])
{'svc__C': [0.1, 1, 10], 'svc__class_weight': ['balanced', None]}


,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_svc__C,param_svc__class_weight,params,split0_test_score,split1_test_score,split2_test_score,...,mean_test_score,std_test_score,rank_test_score,split0_train_score,split1_train_score,split2_train_score,split3_train_score,split4_train_score,mean_train_score,std_train_score
0,0.161344,0.007110,0.017039,0.001560,0.1,balanced,"{'svc__C': 0.1, 'svc__class_weight': 'balanced'}",0.000000,0.000000,0.872727,...,0.523636,0.427547,6,0.000000,0.000000,0.886878,0.911628,0.882883,0.536278,0.437980
1,0.153739,0.009418,0.015586,0.003107,0.1,None,"{'svc__C': 0.1, 'svc__class_weight': None}",0.892857,0.892857,0.872727,...,0.880779,0.009862,4,0.877828,0.877828,0.882883,0.882883,0.882883,0.880861,0.002476
2,0.162538,0.011159,0.015756,0.002518,1,balanced,"{'svc__C': 1, 'svc__class_weight': 'balanced'}",0.943396,0.961538,0.784314,...,0.899906,0.062245,3,0.984293,0.989583,1.000000,0.989691,0.979167,0.988547,0.006923
3,0.130489,0.015290,0.012166,0.001995,1,None,"{'svc__C': 1, 'svc__class_weight': None}",0.892857,0.892857,0.872727,...,0.880779,0.009862,4,0.923810,0.928230,0.946860,0.933333,0.924528,0.931352,0.008457
4,0.130646,0.006456,0.013300,0.002095,10,balanced,"{'svc__C': 10, 'svc__class_weight': 'balanced'}",0.943396,0.925926,0.830189,...,0.916518,0.045161,1,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000
5,0.125979,0.010466,0.012013,0.002648,10,None,"{'svc__C': 10, 'svc__class_weight': None}",0.943396,0.925926,0.830189,...,0.916518,0.045161,1,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000


athaliana carboxylic acid anion inorganic anion
cd-hit: clustered 127 sequences into 90 clusters at threshold 70
label
inorganic anion          46
carboxylic acid anion    44
Name: count, dtype: int64
Pipeline(steps=[('standardscaler', StandardScaler()),
                ('svc', SVC(probability=True, random_state=1))])
{'svc__C': [0.1, 1, 10], 'svc__class_weight': ['balanced', None]}


,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_svc__C,param_svc__class_weight,params,split0_test_score,split1_test_score,split2_test_score,...,mean_test_score,std_test_score,rank_test_score,split0_train_score,split1_train_score,split2_train_score,split3_train_score,split4_train_score,mean_train_score,std_train_score
0,0.028692,0.003344,0.006267,0.000167,0.1,balanced,"{'svc__C': 0.1, 'svc__class_weight': 'balanced'}",0.695652,0.695652,0.000000,...,0.278261,0.340799,6,0.674419,0.674419,0.000000,0.000000,0.000000,0.269767,0.330396
1,0.029498,0.003226,0.006781,0.000474,0.1,None,"{'svc__C': 0.1, 'svc__class_weight': None}",0.695652,0.695652,0.666667,...,0.678261,0.014200,5,0.674419,0.674419,0.681818,0.681818,0.681818,0.678858,0.003625
2,0.031447,0.000984,0.007212,0.000839,1,balanced,"{'svc__C': 1, 'svc__class_weight': 'balanced'}",0.736842,0.615385,0.833333,...,0.732135,0.072054,3,0.982456,1.000000,1.000000,0.983051,0.966667,0.986435,0.012539
3,0.023575,0.003911,0.005978,0.000343,1,None,"{'svc__C': 1, 'svc__class_weight': None}",0.736842,0.571429,0.769231,...,0.691691,0.068776,4,0.982456,1.000000,1.000000,0.983051,0.966667,0.986435,0.012539
4,0.026823,0.002321,0.005072,0.000848,10,balanced,"{'svc__C': 10, 'svc__class_weight': 'balanced'}",0.823529,0.615385,0.769231,...,0.736652,0.071166,1,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000
5,0.025955,0.002274,0.005104,0.000981,10,None,"{'svc__C': 10, 'svc__class_weight': None}",0.823529,0.615385,0.769231,...,0.736652,0.071166,1,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000


## Comparisons

Compare training results with: 

- Average sequence similarity
- GO term similarity
  - How many protein in common?
  - Semantic similarity?
